In [1]:
import os, sys
from pyspark.sql import SparkSession, functions as F, types as T

In [7]:
def spark():
    return (SparkSession.builder
            .appName("silver_clean")
            .config("spark.sql.execution.arrow.pyspark.enabled","true")
            .master("spark://spark-master:7077")
            .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33")
            .config("spark.executor.memory", "2g")
            .config("spark.executor.memoryOverhead", "512m")
            .getOrCreate()
           )

In [8]:
s = spark()


# Extract from mysql

In [ ]:
df = (
    spark.read.format("jdbc")
    .option("url","jdbc:mysql://mysql-container:3307/whole_corp")
    .option("dbtable","whole_corp")
    .option("user", "user").option("password", "pass")
    .load()
)